# 3.1 Parabolic model problem
We are solving the unsteady heat equation 

$$\text{find } u:[0,T] \to H_{0,D}^1 \quad \int_{\Omega} \partial_t u v + \int_{\Omega} \nabla u \nabla v + b \cdot \nabla u v = \int f v  \quad \forall v \in H_{0,D}^1, \quad u(t=0) = u_0$$

with a suitable advective field $b$ (the wind).

In [1]:
from netgen import gui
from math import pi
from ngsolve import *
from netgen.geom2d import SplineGeometry

* Geometry: $(-1,1)^2$
* Dirichlet boundaries everywhere
* Mesh

In [2]:
geo = SplineGeometry()
geo.AddRectangle( (-1, -1), (1, 1), 
                 bcs = ("bottom", "right", "top", "left"))
mesh = Mesh( geo.GenerateMesh(maxh=0.25))
Draw(mesh)
fes = H1(mesh, order=3, dirichlet="bottom|right|left|top")

u,v = fes.TnT()

time = 0.0
dt = 0.001

We define the field $b$ (the wind) as 

$$b(x,y) := (2y(1-x^2),-2x(1-y^2)).$$

In [3]:
b = CoefficientFunction((2*y*(1-x*x),-2*x*(1-y*y)))
Draw(b,mesh,"wind")
from ngsolve.internal import visoptions
visoptions.scalfunction = "wind:0"

* bilinear forms for 
 * convection-diffusion stiffness and 
 * mass matrix seperately.
* non-symmetric memory layout for the mass matrix so that a and m have the same sparsity pattern.

In [4]:
a = BilinearForm(fes, symmetric=False)
a += 0.01*grad(u)*grad(v)*dx + b*grad(u)*v*dx
a.Assemble()

m = BilinearForm(fes, symmetric=False)
m += u*v*dx
m.Assemble()

## Implicit Euler method
$$
  \underbrace{(M + \Delta t A)}_{M^\ast} u^{n+1} = M u^n + \Delta tf^{n+1}
$$

or in an incremental form:

$$
  M^\ast (u^{n+1} - u^n) = - \Delta t A u^n + \Delta tf^{n+1}.
$$

* Incremental form: $u^{n+1} - u^n$ has homogeneous boundary conditions (unless boundary conditions are time-dependent).
* For the time stepping method: set up linear combinations of matrices.
* (Only) if the sparsity pattern of the matrices agree we can copy the pattern and sum up the entries.

First, we create a matrix of the same size and sparsity pattern as m.mat:

In [5]:
mstar = m.mat.CreateMatrix()
print(m.mat.nze, a.mat.nze, mstar.nze)
print(mstar)

11257 11257 11257
Row 0:   0: 0.00520833   4: 0.00260417   31: 0.00260417   89: -0.000520833   90: 8.68056e-05   91: -0.000520833   92: 8.68056e-05   111: -0.000260417   112: -1.32137e-19   553: 0.000173611
Row 1:   1: 0.00876989   10: 0.00223107   11: 0.00215387   38: 0.00438494   93: -0.000446214   94: 7.43691e-05   95: -0.000430774   96: 7.17957e-05   97: -0.000876989   98: 0.000146165   147: -0.000223107   148: 1.13502e-19   151: -0.000215387   152: -1.10114e-19   565: 0.000148738   567: 0.000143591
Row 2:   2: 0.00520833   17: 0.00260417   18: 0.00260417   99: -0.000520833   100: 8.68056e-05   101: -0.000520833   102: 8.68056e-05   185: -0.000260417   186: 1.32137e-19   580: 0.000173611
Row 3:   3: 0.00830036   24: 0.00210642   25: 0.00204376   51: 0.00415018   103: -0.000421283   104: 7.02139e-05   105: -0.000408752   106: 6.81254e-05   107: -0.000830036   108: 0.000138339   227: -0.000210642   228: 1.07573e-19   231: -0.000204376   232: -1.03338e-19   592: 0.000140428   594: 0.0

To access the entries we use the vector of nonzero-entries:

In [6]:
print(mstar.nze)
print(len(mstar.AsVector()))

11257
11257


In [7]:
print(mstar.AsVector())

 0.00520833
 0.00260417
 0.00260417
 -0.000520833
 8.68056e-05
 -0.000520833
 8.68056e-05
 -0.000260417
 -1.32137e-19
 0.000173611
 0.00876989
 0.00223107
 0.00215387
 0.00438494
 -0.000446214
 7.43691e-05
 -0.000430774
 7.17957e-05
 -0.000876989
 0.000146165
 -0.000223107
 1.13502e-19
 -0.000215387
 -1.10114e-19
 0.000148738
 0.000143591
 0.00520833
 0.00260417
 0.00260417
 -0.000520833
 8.68056e-05
 -0.000520833
 8.68056e-05
 -0.000260417
 1.32137e-19
 0.000173611
 0.00830036
 0.00210642
 0.00204376
 0.00415018
 -0.000421283
 7.02139e-05
 -0.000408752
 6.81254e-05
 -0.000830036
 0.000138339
 -0.000210642
 1.07573e-19
 -0.000204376
 -1.03338e-19
 0.000140428
 0.000136251
 0.00260417
 0.0176694
 0.00262878
 0.00620594
 0.00623056
 -0.000520833
 -8.68056e-05
 -0.000260417
 -1.40607e-19
 -0.000525757
 8.76261e-05
 -0.00124119
 0.000206865
 -0.00124611
 0.000207685
 -0.000262878
 -1.37219e-19
 -0.000360177
 1.91429e-19
 0.000173611
 0.000175252
 0.000240118
 0.00262878
 0.0158516
 0.00250

Using the vector we can build the linear combination of the a and the m matrix:

In [8]:
mstar = m.mat.CreateMatrix()
mstar.AsVector().data = m.mat.AsVector() + dt * a.mat.AsVector()
invmstar = mstar.Inverse(freedofs=fes.FreeDofs())

We set the r.h.s. $f = exp(-6 ((x+\frac12)^2+y^2)) - exp(-6 ((x-\frac12)^2+y^2))$

In [9]:
f = LinearForm(fes)
gaussp = exp(-6*((x+0.5)*(x+0.5)+y*y))-exp(-6*((x-0.5)*(x-0.5)+y*y))
Draw(gaussp,mesh,"f")
f += gaussp*v*dx
f.Assemble()

and the initial data: $u_0 = (1-y^2)x$

In [10]:
gfu = GridFunction(fes)
gfu.Set((1-y*y)*x)
Draw(gfu,mesh,"u")

In [11]:
res = gfu.vec.CreateVector()
tstep = 1 # time that we want to step over within one block-run

In [16]:
%%time
t_intermediate=0 # time counter within one block-run
while t_intermediate < tstep - 0.5 * dt:
    res.data = dt * f.vec - dt * a.mat * gfu.vec
    gfu.vec.data += invmstar * res
    t_intermediate += dt
    print("\r",time+t_intermediate,end="")
    Redraw(blocking=True)
print("")
time+=t_intermediate

 5.0000000000000036
CPU times: user 1.29 s, sys: 235 ms, total: 1.53 s
Wall time: 1.67 s


### Alternative version with iterative solvers

* For a factorization of $M^\ast$ (${M^\ast}^{-1}$) we require a sparse matrix $M^\ast$ 
* To store $M^\ast$ as a sparse matrix requires new storage (and same memory layout of $A$ and $M$)
* For iterative solvers we only require the matrix (and preconditioner) applications
* `mstar = m.mat + dt * a.mat` has no storage but matrix-vector multiplications

iterative solver version (with Jacobi preconditining):

In [ ]:
mstar_alt = m.mat + dt * a.mat
premstar_alt = m.mat.CreateSmoother() + dt * a.mat.CreateSmoother()
invmstar_alt = CGSolver(mstar_alt,premstar_alt,printrates=True)

print(premstar_alt)

In [ ]:
%%time
t_intermediate=0 # time counter within one block-run
while t_intermediate < tstep - 0.5 * dt:
    res.data = dt * f.vec - dt * a.mat * gfu.vec
    gfu.vec.data += invmstar_alt * res
    t_intermediate += dt
    print("\r t = {:24}, iteration steps: {}".format(time+t_intermediate,invmstar_alt.GetSteps()), end="")
    Redraw(blocking=True)
print("")
time+=t_intermediate

## Supplementary material:
* [Time dependent r.h.s.](#TDRHS)
* [Time dependent boundary data](#TDBND)
* [Runge-Kutta time integration](#RK)
* [VTK Output](#VTK)

## Supplementary 1: time-dependent r.h.s. data <a class="anchor" id="TDRHS"></a>

Next: time-dependent r.h.s. data $f=f(t)$:

* Use `Parameter` t representing the time. 
* A `Parameter` is a constant `CoefficientFunction` the value of which can be changed with the `Set`-function.

In [ ]:
t = Parameter(0.0)

An example of a time-dependent coefficient that we want to use as r.h.s. in the following is

In [ ]:
omega=1
gausspt = exp(-6*((x+sin(omega*t))*(x+sin(omega*t))+y*y))-exp(-6*((x-sin(omega*t))*(x-sin(omega*t))+y*y))
Draw(gausspt,mesh,"ft",sd=4)
time = 0.0
while time < 10 - 0.5 * dt:
    t.Set(time)
    Redraw(blocking=True)
    time += 1e-5

Accordingly we define a different linear form which then has to be assembled in every time step.

In [ ]:
ft = LinearForm(fes)
ft += gausspt*v*dx
time = 0.0
t.Set(0.0)
gfu.Set((1-y*y)*x)
#gfu.Set(CoefficientFunction(0))
Draw(gfu,mesh,"u")

In [ ]:
tstep = 10 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
res = gfu.vec.CreateVector()
while t_intermediate < tstep - 0.5 * dt:
    t.Set(time+t_intermediate+dt)
    ft.Assemble()
    res.data = dt * ft.vec - dt * a.mat * gfu.vec
    gfu.vec.data += invmstar * res
    t_intermediate += dt
    print("\r",time+t_intermediate,end="")
    Redraw(blocking=True)
print("")
time+=t_intermediate

## Supplementary 2: Time dependent boundary conditions <a class="anchor" id="TDBND"></a>


* $u|_{\partial \Omega} = u_D(t)$, $f=0$
* implicit Euler time stepping method, non-incremental form:

  $$
    M^\ast u^{n+1} = (M + \Delta t A) u^{n+1} = M u^n
  $$  
  
* Homogenize w.r.t. to boundary conditions, i.e. we split 

  $$ u^{n+1} = u^{n+1}_0 + u^{n+1}_D $$ 
  
  where $u^{n+1}_D$ is a (discrete) function with correct boundary condition:  
  
$$
  {M^\ast} u^{n+1}_0 = M u^n - {M^\ast} u^{n+1}_D
$$


In [ ]:
uD = CoefficientFunction( [(1-x*x)*IfPos(sin(0.3*pi*t),sin(0.3*pi*t),0),0,0,0])
time = 0.0
t.Set(0.0)
gfu.Set(uD,BND)
Draw(gfu,mesh,"u")
# visualization stuff
from ngsolve.internal import *
visoptions.mminval = 0.0
visoptions.mmaxval = 0.2
visoptions.deformation = 0
visoptions.autoscale = 0

In [ ]:
tstep = 2 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
res = gfu.vec.CreateVector()
while t_intermediate < tstep - 0.5 * dt:
    t.Set(time+t_intermediate+dt)
    res.data = m.mat * gfu.vec
    gfu.Set(uD,BND)
    res.data -= mstar * gfu.vec
    gfu.vec.data += invmstar * res
    t_intermediate += dt
    print("\r",time+t_intermediate,end="")
    Redraw(blocking=True)
print("")
time+=t_intermediate

## Supplementary 3: Singly diagonally implicit Runge-Kutta methods <a class="anchor" id="RK"></a>


We consider more sophisticated time integration methods, SDIRK methods. To simplify presentation we set $f=0$.

SDIRK methods for the semi-discrete problem $\frac{d}{dt} u = M^{-1} F(u) = -M^{-1} \cdot A u$ are of the form:

$$
  u^{n+1} = u^n + \Delta t M^{-1} \sum_{i=0}^{s-1} b_i k_i
$$

with

$$
k_i = - A u_i \quad \text{ where $u_i$ is the solution to } \quad 
(M + a_{ii} \Delta t A) u_i = M u^n - \Delta t \sum_{i=0}^{i-1} a_{ij} k_j, \quad i=0,..,s-1.
$$

The coefficients a,b and c are stored in the butcher tableau:

$$
\begin{array}{c|cccc}
c_0 & a_{00} & 0 & \ddots& \\
c_1 & a_{10} & a_{11} & 0 & \ddots \\
\vdots & \vdots & \vdots & \ddots & 0\\
c_{s-1} & a_{s-1,0} & a_{s-1,1} & \dots& a_{s-1,s-1} \\ \hline
 & b_{0} & b_1 & \dots& b_{s-1} \\
\end{array}
$$

For an SDIRK method we have $a^\ast = a_{ii},~i=0,..,s-1$.

Simplest example: Implicit Euler
$$
\begin{array}{c|c}
1 & 1 \\ \hline
 & 1  \\
\end{array}
$$

In [ ]:
class sdirk1: #order 1 (implicit Euler)
    stages = 1
    a = [[1]]
    b = [1]
    c = [1]
    astar = 1

We can use for example the 2 stage SDIRK (order 3) method

$$
\begin{array}{c|cc}
p & p & 0 \\
1-p & 1-2p & p \\ \hline
 & 1/2 & 1/2 \\
\end{array}
$$

with $p = \frac{3 - \sqrt{3}}{6}$.

In [ ]:
class sdirk2: #order 3
    p = (3 - sqrt(3))/6
    stages = 2
    a = [[p, 0], 
         [1 - 2*p, p]]
    b = [1/2, 1/2]
    c = [p, 1 - p]
    astar = p
    

We can use for example the 5 stage SDIRK (order 4) method
$$
\begin{array}{c|cccc}
1/4 & 1/4  \\
3/4 & 1/2 & 1/4 & \\
11/20 & 17/50 & -1/25 & 1/4 \\
1/2 & 371/1360 & -137/2720 & 15/544 & 1/4 \\
1 & 25/4 & -49/48 & 125/16 & -85/12 & 1/4 \\ \hline
 & 25/4 & -49/48 & 125/16 & -85/12 & 1/4 \\
\end{array}
$$

In [ ]:
class sdirk5: #order 4
    stages = 5
    a = [[1/4, 0, 0, 0, 0], 
         [1/2, 1/4, 0, 0, 0], 
         [17/50,-1/25, 1/4, 0, 0],
         [371/1360, -137/2720, 15/544, 1/4,0],
         [25/24, -49/48, 125/16, -85/12, 1/4]]
    b = [25/24, -49/48, 125/16, -85/12, 1/4]
    c = [1/4, 3/4, 11/20, 1/2, 1]
    astar = 1/4    

### SDIRK2 :

In [ ]:
butchertab = sdirk2()   
rhsi = gfu.vec.CreateVector()
Mu0 = gfu.vec.CreateVector()
ui = gfu.vec.CreateVector()
k = [gfu.vec.CreateVector() for i in range(butchertab.stages)]

We have to update the $M^\ast$ matrix and reset initial data

In [ ]:
time = 0.0
t.Set(0.0)
gfu.Set(uD,BND)
Draw(gfu,mesh,"u")
# visualization stuff
from ngsolve.internal import *
visoptions.mminval = 0.0
visoptions.mmaxval = 0.2
visoptions.deformation = 0
visoptions.autoscale = 0

mstar.AsVector().data = m.mat.AsVector() + butchertab.astar * dt * a.mat.AsVector()
invmstar = mstar.Inverse(freedofs=fes.FreeDofs())
invmass = m.mat.Inverse(freedofs=fes.FreeDofs())

In [ ]:
tstep = 2 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
while t_intermediate < tstep - 0.5 * dt:
    Mu0.data = m.mat * gfu.vec
    for i in range(butchertab.stages):
        # add up the ks as prescribed in the butcher tableau
        rhsi.data = Mu0
        for j in range(0,i):
            rhsi.data += dt * butchertab.a[i][j] * k[j]
        # Solve for ui (with homogenization for the boundary data)
        t.Set(time+t_intermediate+butchertab.c[i]*dt)
        gfu.Set(uD,BND)
        ui.data = gfu.vec; rhsi.data -= mstar * ui
        ui.data += invmstar * rhsi
        # compute k[i] from ui
        k[i].data = - a.mat * ui
    t_intermediate += dt; t.Set(time+t_intermediate)
    # Adding up the ks:
    gfu.Set(uD,BND)
    Mu0.data -= m.mat * gfu.vec
    for i in range(0,butchertab.stages):
        Mu0.data += dt * butchertab.b[i] * k[i]
    gfu.vec.data += invmass * Mu0        
    print("\r",time+t_intermediate,end="")
    Redraw(blocking=True)
print(""); time+=t_intermediate            

## Supplementary 4: VTK Output ("exporting the nice pictures") <a class="anchor" id="VTK"></a>

* see also https://ngsolve.org/blog/ngsolve/2-vtk-output
* or `py_tutorials/vtkout.py` (ngsolve repository)

Outputting the nice pictures to vtk (to visualize with paraview):

In [ ]:
vtk = VTKOutput(ma=mesh,coefs=[gfu],
                names=["sol"],
                filename="vtk_example1",
                subdivision=3)
vtk.Do()

In [ ]:
%%bash
ls vtk_example1.vtk

You can also export vector fields:

In [ ]:
vtk = VTKOutput(ma=mesh,coefs=[gfu,grad(gfu)],names=["sol","gradsol"],filename="vtk_example2",subdivision=3)
vtk.Do()

In [ ]:
#%%bash
#paraview vtk_example2.vtk

And time dependent data:

In [ ]:
vtk = VTKOutput(ma=mesh,coefs=[gfu],names=["sol"],filename="vtk_example3",subdivision=3)
vtk.Do()
gfu.Set((1-y*y)*x)
time = 0
tstep = 1 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
res = gfu.vec.CreateVector()
i = 0
while t_intermediate < tstep - 0.5 * dt:
    res.data = dt * f.vec - dt * a.mat * gfu.vec
    gfu.vec.data += invmstar * res
    t_intermediate += dt
    print("\r",time+t_intermediate,end="")
    Redraw(blocking=True)
    i += 1
    if (i%10 == 0):
        vtk.Do()
print("")

In [ ]:
#%%bash
#paraview vtk_example3_..vtk